In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="mlflow")

In [3]:
import mlflow

mlflow.set_tracking_uri("http://localhost:8090")
# mlflow.set_tracking_uri("sqlite:///mlflow.db") # Local Database
mlflow.set_experiment("rightmove-rent-prediction")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1706524695369, experiment_id='1', last_update_time=1706524695369, lifecycle_stage='active', name='rightmove-rent-prediction', tags={}>

# Load Data

In [4]:
from mlflow.data.pandas_dataset import PandasDataset
train_dataset_source_url = "data/train.csv"
val_dataset_source_url = "data/val.csv"

train_df = pd.read_csv(train_dataset_source_url, index_col=0)
val_df = pd.read_csv(val_dataset_source_url, index_col=0)

In [5]:
train_df.head()

,bedrooms,bathrooms,price,commercial,development,students,longitude,latitude,text,walk_score,restaurants,shopping,coffee,banks,parks,schools,books,entertainment,grocery
38830,3,1.0,24000.0,0,0,0,-0.056192,51.667706,"THREE BEDROOM HOUSE, TWO RECEPTION ROOMS, FITT...",15.133139,3.179574e-01,0.000000e+00,5.457925e-08,2.781163e-09,0.992856,0.823671,2.903329e-07,6.413508e-45,0.134351
55467,1,1.0,19800.0,0,0,0,-0.110176,51.568771,", Hunters Stanmore are proud to market this o...",85.781982,2.999974e+00,8.963245e-01,1.999985e+00,0.000000e+00,0.999051,0.991928,9.999584e-01,9.736460e-01,3.000000
39597,3,1.0,10500.0,0,0,0,-1.426080,53.687050,"Well Presented, Semi Detached, Unfurnished, Th...",17.884093,1.087511e-08,1.955617e-14,3.639962e-09,7.589710e-09,0.062920,0.287689,2.679829e-07,0.000000e+00,2.330664
58613,2,1.0,11400.0,0,0,0,-4.259540,55.820202,"Mod Ground Floor Flat, 2 Bedrooms, Unfurnished...",64.970217,1.795519e+00,1.638024e-05,1.809908e+00,6.343372e-01,0.991111,0.910470,5.993718e-01,0.000000e+00,2.999929
25056,3,NaN,48000.0,0,0,0,-0.736478,51.425994,"Garden, Cooking basics, Dedicated workspace, W...",41.593268,3.870290e-06,1.016736e-19,1.092181e+00,1.785391e-15,0.895333,0.999893,8.105341e-01,1.367258e-30,2.437928


In [6]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [7]:
def create_dataset(train_df, val_df, features):
    X_train = train_df[features]
    y_train = train_df[['price']]
    
    X_val = val_df[features]
    y_val = val_df[['price']]

    train_dataset: PandasDataset = mlflow.data.from_pandas(train_df[features], source=train_dataset_source_url)
    val_dataset: PandasDataset = mlflow.data.from_pandas(val_df[features], source=val_dataset_source_url)

    return X_train, y_train, X_val, y_val, train_dataset, val_dataset

# Linear Regression

## Numerical features

In [8]:
numerical_features = ['bedrooms', 'bathrooms', 'longitude', 'latitude']

In [51]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features)

In [52]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "Linear Regression")
    mlflow.log_param("Included Data", numerical_features)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

## Walk Score

In [13]:
numerical_features_walk_score = ['bedrooms', 'bathrooms', 'longitude', 'latitude', 'walk_score']

In [54]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score)

In [55]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "Linear Regression")
    mlflow.log_param("Included Data", numerical_features_walk_score)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

## Binary features

In [11]:
numerical_features_walk_score_binary = ['bedrooms', 'bathrooms', 'longitude', 'latitude', 'walk_score', 'commercial', 'development', 'students']

In [57]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score_binary)

In [58]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "Linear Regression")
    mlflow.log_param("Included Data", numerical_features_walk_score_binary)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

In [59]:
rmse

17936.675239064596

# Random Forest

In [62]:
from sklearn.ensemble import RandomForestRegressor

## Numerical

In [60]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features)

In [63]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "Random Forest")
    mlflow.log_param("Included Data", numerical_features)
    
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


## Walk score

In [65]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score)

In [66]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "Random Forest")
    mlflow.log_param("Included Data", numerical_features_walk_score)
    
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


## Binary values

In [67]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score_binary)

In [68]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "Random Forest")
    mlflow.log_param("Included Data", numerical_features_walk_score_binary)
    
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


# XGBoost

In [69]:
import xgboost as xgb

In [70]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features)

In [71]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "XGBoost")
    mlflow.log_param("Included Data", numerical_features)
    
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

## Walk score

In [73]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score)

In [74]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "XGBoost")
    mlflow.log_param("Included Data", numerical_features_walk_score)
    
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

In [75]:
rmse

20303.760748490633

In [76]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score_binary)

In [77]:
with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "XGBoost")
    mlflow.log_param("Included Data", numerical_features_walk_score_binary)
    
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

# Finetune XGBoost

In [79]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [81]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score_binary)

In [82]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [84]:
def objective(params):
    with mlflow.start_run(): # Start a new MLflow run for each evaluation
        mlflow.set_tag("Model type", "XGBoost") # Log model tag
        mlflow.log_params(params) # Log parameters

        # Train the model
        booster = xgb.train(
            params=params,
            dtrain=train, # Your training dataset
            num_boost_round=1000,
            evals=[(valid, 'validation')], # Your validation dataset
            early_stopping_rounds=50
        )

        # Make predictions and calculate RMSE
        y_pred = booster.predict(valid) # Your validation dataset
        rmse = mean_squared_error(y_val, y_pred, squared=False) # y_val should be your validation target
        mlflow.log_metric("rmse", rmse) # Log RMSE metric

    return {'loss': rmse, 'status': STATUS_OK}

In [85]:
# Define the search space for Hyperopt
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror', # Updated objective for regression
    'seed': 42
}

In [ ]:
# Run Hyperopt
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

print("Best Hyperparameters:", best_result)

In [96]:
best_result['max_depth'] = 94

## Save Random Forest

In [88]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score_binary)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    mlflow.log_params(best_result)

    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "XGBoost")
    mlflow.log_param("Included Data", numerical_features_walk_score_binary)

    booster = xgb.train(
        params=best_result,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # with open("models/preprocessor.b", "wb") as f_out:
    #     pickle.dump(dv, f_out)
    # mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [14]:
X_train, y_train, X_val, y_val, train_dataset, val_dataset = create_dataset(train_df, val_df, numerical_features_walk_score)

In [17]:
from sklearn.ensemble import RandomForestRegressor

with mlflow.start_run():
    # Log the dataset to the MLflow Run. Specify the "training" context to indicate that the
    # dataset is used for model training
    mlflow.log_input(train_dataset, context="training")
    mlflow.log_input(val_dataset, context="validation")
    
    mlflow.set_tag("developer", "Alex") # Addind metadata
    
    mlflow.log_param("Model type", "Random Forest")
    mlflow.log_param("Included Data", numerical_features_walk_score)
    
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

    mlflow.sklearn.log_model(model, artifact_path="random-forest-model")

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
